In [8]:
# load basic libraries
import re
import nltk
import numpy as np
import pandas as pd
from sklearn import naive_bayes
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test_split
from sklearn import neighbors, datasets

import re
import nltk
import operator
import numpy as np
import pandas as pd
import seaborn as sns
import collections as ct

from bs4 import BeautifulSoup
from nltk import pos_tag,word_tokenize
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import naive_bayes ,svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold,train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec



%matplotlib inline

C:\Users\be231\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def ReadFile(inFile):
    with open(inFile, "r") as f:
        content = f.readlines()   
    content = [x.strip() for x in content]
    return (content)

def clean_tweet(tweet):
    #Utility function to clean tweet text by removing links, special characters using simple regex statements.
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())


def Preprocessing(tweet,emotions,positive_Words,negative_Words,stop_Words):
    for emotion in emotions:
        tweet = tweet.replace(emotion[0], " "+emotion[1]+" ",100)
    #print (tweet)
    buff = ''
    for word in tweet.split():
        if word not in stop_Words:
            if word in positive_Words:
                buff += "positive "
                continue
            elif word in negative_Words:
                buff += "negative "
                continue
            else:
                buff += nltk.stem.porter.PorterStemmer().stem(word)+" "
    tweet = buff
    tweet = clean_tweet(tweet)
    
    
    return tweet

In [3]:
priPreprocessedPD = pd.read_csv("sumTraining.csv",names=['ID','Type','Body'])
priPreprocessedPD.head()
#print ("Positive instances:" + str(len(np.where(priPreprocessedPD['Type'] == 'positive')[0])))
#print ("Negative instances:" + str(len(np.where(priPreprocessedPD['Type'] == 'negative')[0])))
#print ("Neutral instances:" + str(len(np.where(priPreprocessedPD['Type'] == 'neutral')[0])))
priPreprocessedPD.head(10)

,ID,Type,Body
0,2.640000e+17,positive,I haven't money
1,2.630000e+17,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,2.620000e+17,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,2.640000e+17,negative,Iranian general says Israel\u2019s Iron Dome c...
4,2.630000e+17,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...
5,2.640000e+17,neutral,I sat through this whole movie just for Harry ...
6,2.640000e+17,positive,with J Davlar 11th. Main rivals are team Polan...
7,2.640000e+17,negative,Talking about ACT\u2019s && SAT\u2019s\u002c d...
8,2.120000e+17,neutral,"Why is \""""Happy Valentines Day\"""" trending? It..."
9,2.550000e+17,negative,They may have a SuperBowl in Dallas\u002c but ...


In [5]:
priPreprocessedPD.head(10)

,ID,Type,Body
0,2.640000e+17,positive,I haven't money
1,2.630000e+17,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,2.620000e+17,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,2.640000e+17,negative,Iranian general says Israel\u2019s Iron Dome c...
4,2.630000e+17,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...
5,2.640000e+17,neutral,I sat through this whole movie just for Harry ...
6,2.640000e+17,positive,with J Davlar 11th. Main rivals are team Polan...
7,2.640000e+17,negative,Talking about ACT\u2019s && SAT\u2019s\u002c d...
8,2.120000e+17,neutral,"Why is \""""Happy Valentines Day\"""" trending? It..."
9,2.550000e+17,negative,They may have a SuperBowl in Dallas\u002c but ...


In [11]:
tweets_train,tweets_test,train_labels,test_labels = train_test_split(priPreprocessedPD["Body"],priPreprocessedPD['Type'], test_size=0.2)
tweets_test.head()

10438    @eckoxsoldier For some reason Amazon app store...
3251     \u2019Now Behold the Lamb\u002c the precious L...
7935     #ESIAfrica Wescoal signs three year supply agr...
9894     @sunnyvegas702 Nice! Disagreement! Of course, ...
3344     Mee& my grandpa are going to the rita game tom...
Name: Body, dtype: object

In [12]:
kf = StratifiedKFold(n_splits=10, shuffle=True)
arr = []
for train_idx, val_idx in kf.split(tweets_train, train_labels):
    train_data = tweets_train.iloc[train_idx]
    train_label = train_labels.iloc[train_idx]
    val_data = tweets_train.iloc[val_idx]
    val_label = train_labels.iloc[val_idx]
    vectorizer = CountVectorizer()#TfidfVectorizer()#
    X_train = vectorizer.fit_transform(train_data)
    X_val = vectorizer.transform(val_data)
    #clfr = naive_bayes.MultinomialNB()
    #clfr = svm.SVC()
    #clfr = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
    clfr = neighbors.KNeighborsClassifier(15)
    clfr.fit(X_train,train_label)
    print ('#')

tweets_test_vectorized = vectorizer.transform( tweets_test)
predicted = clfr.predict(tweets_test_vectorized)
acc = metrics.accuracy_score(test_labels,predicted)
print (metrics.classification_report(test_labels,predicted))
print('accuracy = '+str(acc*100)+'%')

#
#
#
#
#
#
#
#
#
#
             precision    recall  f1-score   support

   negative       0.40      0.02      0.03       471
    neutral       0.49      0.49      0.49      1384
   positive       0.46      0.62      0.53      1354

avg / total       0.47      0.47      0.44      3209

accuracy = 47.4602679963%


In [17]:
clfrNaive = naive_bayes.MultinomialNB()
clfrNaive.fit(X_train,train_label)
predictedNaive = clfrNaive.predict(tweets_test_vectorized)
acc = metrics.accuracy_score(test_labels,predictedNaive)
print (metrics.classification_report(test_labels,predictedNaive))
print('accuracy = ',acc*100,'%')

             precision    recall  f1-score   support

   negative       0.67      0.15      0.24       471
    neutral       0.63      0.60      0.61      1384
   positive       0.59      0.79      0.68      1354

avg / total       0.62      0.61      0.59      3209

accuracy =  61.140542225 %


In [24]:
clr =  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
x = LogisticRegression()
clfr.fit(X_train,train_label)
predicted = clfr.predict(tweets_test_vectorized)
acc = metrics.accuracy_score(test_labels,predicted)
print (metrics.classification_report(test_labels,predicted))
print('accuracy = ',acc*100,'%')

             precision    recall  f1-score   support

   negative       0.40      0.02      0.03       471
    neutral       0.49      0.49      0.49      1384
   positive       0.46      0.62      0.53      1354

avg / total       0.47      0.47      0.44      3209

accuracy =  47.4602679963 %


In [25]:
#Logestic regression
pipe = Pipeline([('classifier', LogisticRegression())])
search_space = [{'classifier': [LogisticRegression()],
                 'classifier__C': [0.001,0.01,0.1,1,10,100],
                'classifier__n_jobs':[-1],
                'classifier__multi_class':['ovr']}]
               

clf_gss = GridSearchCV(pipe, search_space, cv=5, verbose=0)

best_model = clf_gss.fit(X_train,train_label)

# View best model
print (best_model.best_estimator_.get_params()['classifier'])

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [ ]:
pipe = Pipeline([('classifier', RandomForestClassifier())])
search_space = [{'classifier': [RandomForestClassifier()],
                 'classifier__n_estimators': [10, 100, 1000],
                 'classifier__max_features': [1, 2, 3]}]
               

clf_gss = GridSearchCV(pipe, search_space, cv=5, verbose=0)

best_model = clf_gss.fit(X_train,train_label)

# View best model
print (best_model.best_estimator_.get_params()['classifier'])